<a href="https://colab.research.google.com/github/LTopaloglou/transformer/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
from torch.torch_version import Version
import torch
import torch.nn as nn
from torch.nn import functional as F

#I am using variable names defined in "Attention Is All You Need" for simplicity.

#Define a single Scaled Dot-Product Attention head. I am simplifying it by enforcing d_k = d_v
class Attention(nn.Module):

  def __init__(self, d_k, mask):
    super().__init__()
    self.mask = mask
    self.d_k = d_k
    self.register_buffer('lowerTriangle', torch.tril(torch.ones(d_k, d_k)))

  def forward(self, Q, K, V):
    #Takes in vectors of Queries, Keys & Values. Queries, Keys & Values have dimension d_k x 1 (one dimensional vectors)
    if (Q.shape[0] != self.d_k or K.shape[0] != self.d_k or V.shape[0] != self.d_k):
      raise Exception('Invalid Query, Key or Value Dimensions')
    #First take the dot product of Queries & Keys
    weight = Q @ K.transpose(0, 1)
    #Now scale by 1/sqrt(d_k)
    weight = weight  * (self.d_k**-0.5)
    #Mask everything not in the lower triangular of weight
    if (self.mask):
      weight = weight.masked_fill(self.lowerTriangle== 0, float('-inf'))
    #Now apply softmax in the dimension of the rows
    weight = weight.softmax(1)
    #Finally, multiply values by weights to get the outputs
    out = weight @ V # [d_k, d_k] x [d_k, 1] = [d_k, 1]
    return out

#Define a Multi-Head Attention layer. As in the paper, I am setting d_k = d_model/h
class MultiHeadAttention(nn.Module):

  def __init__(self, d_model, h, mask):
    super().__init__()
    self.mask = mask
    self.d_model = d_model
    self.h = h
    self.d_k = int(d_model/h)
    if (self.d_k != d_model/h):
      raise Exception('Invalid Dimensions Provided') #Ensure valid dimensions
    self.W_O = nn.Linear(h*self.d_k, d_model, bias=False) #Output linear layer
    self.W_Q = nn.ModuleList() #The h different Query linear layers
    self.W_K = nn.ModuleList() #The h different Key linear layers
    self.W_V = nn.ModuleList() #The h different Value linear layers
    self.Att = Attention(self.d_k, mask) #I think (?) only 1 attention layer is needed since no backprop happens TODO: VERIFY
    for i in range(h):
      #Initialize all h linear layers for Q, K, V
      self.W_Q.append(nn.Linear(d_model, self.d_k, bias=False))
      self.W_K.append(nn.Linear(d_model, self.d_k, bias=False))
      self.W_V.append(nn.Linear(d_model, self.d_k, bias=False)) #TODO: Instead of Linear layers, just create Matrices

  def forward(self, Q, K, V):
    #The inputs are Queries, Keys and Vectors which each have size d_model x 1
    if (Q.shape[0] != self.d_model or K.shape[0] != self.d_model or V.shape[0] != self.d_model):
      raise Exception('Invalid Query, Key or Value Dimensions')
    heads = []
    for i in range(self.h):
      queries = self.W_Q[i].weight(Q)
      keys = self.W_K[i].weight(K)
      values = self.W_V[i].weight(V)
      heads.append(self.Att.forward(Q, K, V))
    out = torch.cat(heads, 0)
    print(out)

In [57]:
#For testing & validation of Attention
att = Attention(5, True)
keys = torch.ones(5, 1)
queries = torch.ones(5, 1)
values = torch.ones(5, 1)
print(att.forward(queries, keys, values))

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])


In [102]:
#For testing & validation of MultiHeadAttention
mult = MultiHeadAttention(16, 4, False)
Q = torch.ones(16, 1)
K = torch.ones(16, 1)
V = torch.ones(16, 1)
#TODO: What is the dimensions of Q, K, V in the paper? Must be a matrix?
print(Q)
mult.forward(Q, K, V)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])


TypeError: ignored